## Розробка виграшної стратегії для гри «хрестики-нулики» 

In [3]:
from typing import Tuple
from IPython.display import display, HTML, Markdown
from abc import ABC, abstractmethod
import random

In [4]:
CROSS = 'X'
ZERO = '0'
BLANK = '_'

Базовий класс для агента в грі

In [5]:
class BaseAgent(ABC):
    def __init__(self, field, is_cross):
        self._field = field
        self._is_cross = is_cross

    @property
    def mark(self):
        return self._is_cross and CROSS or ZERO

    def on_field_changes(self, x, y, is_cross):
        self._filed[x][y] = is_cross and CROSS or ZERO

    @abstractmethod
    def make_step(self) -> Tuple[int, int]:
        pass

Функція перевірки переможця

In [6]:
def is_cross_win(field):
    """
    :return:
        True if cross wins
        False of zero wins
        return None if undefined
    """
    def check(line):
        if line.count(CROSS) == 3:
            return True
        elif line.count(ZERO) == 3:
            return False
        return None

    for row in field:
        if check(''.join(row)) is not None: return check(''.join(row))

    for column_index in range(3):
        line = ''.join(f'{field[i][column_index]}' for i in range(3))
        if check(line) is not None: return check(line)

    d1 = ''.join(f'{field[i][i]}' for i in range(3))
    if check(d1) is not None: return check(d1)
    d2 = ''.join(f'{field[i][2-i]}' for i in range(3))
    if check(d2) is not None: return check(d2)

Реалізація "круп'є" гри

In [7]:
class Game:
    def __init__(self, cross_agent_class, zero_agent_class):
        self._field = [
            [BLANK for __ in range(3)]
            for _ in range(3)
        ]
        self._cross_agent = cross_agent_class(self._field, True)
        self._zero_agent = zero_agent_class(self._field, False)

    def _display_field(self):
        html = "<table>"
        for row in self._field:
            html += "<tr>"
            for field in row:
                html += "<td><h4>{}</h4><td>".format(field)
            html += "</tr>"
        html += "</table>"
        display(HTML(html))

    def start(self):
        step_num = 0
        agents = [
            self._cross_agent,
            self._zero_agent,
        ]
        while True:
            agent = agents[step_num%2]
            display(Markdown(f'## Хід {agent.mark == CROSS and "Хрестика" or "Нулика"}'))
            x,y = agent.make_step()
            self._field[x][y] = agent.mark
            self._display_field()
            win_state = is_cross_win(self._field)
            if win_state is not None and win_state:
                display(Markdown("# Переміг xрестик!"))
                break
            elif win_state is not None and not win_state:
                display(Markdown("# Переміг нулик!"))
                break
            elif ''.join([''.join(row) for row in self._field]).count(BLANK) == 0:
                display(Markdown("# Нічія!!!"))
                break
            step_num+=1            
            display(Markdown('***'))

реалізація агента для забезпечення доступу "живого" гравця до гри

In [8]:
class UserAgent(BaseAgent):
    def make_step(self)-> Tuple[int, int]:
        user_input = input('type row number and col number with space. i.e. `1 2`').split(' ')
        return int(user_input[0]),  int(user_input[1])

реалізація агента гри, ціль якого грати хрестиком і використовувавти макс стратегію

In [32]:
class MaxCrossAgent(BaseAgent):
    _depth = 5

    def make_step(self)-> Tuple[int, int]:
        def copy_field(field):
            return [row.copy() for row in field]
        positions_score = {i:[] for i in range(4,0,-1) }

        def position_generator(field):
            for i in range(9):
                row = int(i/3)
                col = int(i%3)
                if field[row][col] == BLANK:
                    yield row, col

        def score_position(symbol, field, depth, row, col):
            temp_field = copy_field(field)
            temp_field[row][col] = symbol
            win_check = is_cross_win(temp_field)
            if win_check:
                return 4
            if win_check is not None and not win_check:
                return 1
            if depth==0:
                return 3
            possible_position = list(position_generator(temp_field))
            if len(possible_position) == 0:
                return 2
            
            scores =[
                score_position(symbol==CROSS and ZERO or CROSS, temp_field, depth-1, r, c)
                for r, c in possible_position
            ]
            return symbol == ZERO and max(scores) or min(scores)
        for r, c in position_generator(self._field):
            score = score_position(CROSS, self._field, self._depth, r, c)
            positions_score[score].append((r, c))
        print(positions_score)
        for score in  range(4,0,-1):
            if len(positions_score[score])>0:
                result = random.choice(positions_score[score])
                print(result)
                return result



### Демонстрація декількох прикладів гри. 

***
***
# Перемога гравця
***
***

In [33]:
Game(MaxCrossAgent, UserAgent).start()

## Хід Хрестика

{4: [], 3: [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)], 2: [], 1: []}
(1, 2)


***

## Хід Нулика

type row number and col number with space. i.e. `1 2` 0 0


...

type row number and col number with space. i.e. `1 2` 2 0


***

## Хід Хрестика

{4: [(1, 0)], 3: [], 2: [], 1: [(0, 1), (2, 1)]}
(1, 0)


# Переміг xрестик!

***
***
# Перемога агента
***
***

In [46]:
Game(MaxCrossAgent, UserAgent).start()

## Хід Хрестика

***

## Хід Нулика

type row number and col number with space. i.e. `1 2` 1 1


***

...

## Хід Нулика

type row number and col number with space. i.e. `1 2` 1 2


***

## Хід Хрестика

# Переміг xрестик!

***
***
# Нічия
***
***

In [59]:
Game(MaxCrossAgent, UserAgent).start()

## Хід Хрестика

***

## Хід Нулика

type row number and col number with space. i.e. `1 2` 1 1


***

...

## Хід Нулика

type row number and col number with space. i.e. `1 2` 2 0


***

## Хід Хрестика

# Нічія!!!